# USDA-NASS CropScape Data
The following code takes data coordinates and returns a list with information from the crop for the location. This is an experimental notebook so you guys can see what I tried. At the end, I have a function that takes latitude and longitude coordinates and returns a dataframe with the following columns: <b>year,x, y, value, category, color, Log, Lat. </b>

In [1]:
import requests
import urllib3
# I am disabling the annoying warning signs. 
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


#I will try the sample code from 
# https://nassgeodata.gmu.edu/CropScape/devhelp/help.html
#Maybe we should use another method since I turned off the verification of certificates.
response = requests.get('https://nassgeodata.gmu.edu/axis2/services/CDLService/GetCDLValue?year=2018&x=1551565.363&y=1909363.537', verify=False)

In [2]:
import pandas as pd
from bs4 import BeautifulSoup

#getting the data from the results section. 
soup = BeautifulSoup(response.content,'xml')
table = soup.find_all('Result')[0] 
table

<Result>{x: 1551565.363, y: 1909363.537, value: 1, category: "Corn", color: "#FFD400"}</Result>

In [3]:
import re    
#cleaning the data. 
reg = str(table)
reg = re.findall('{(.+?)}', reg)

my_list = str(reg).replace("'", "")
my_listp = str(my_list).replace("[", "")
my_listp = str(my_listp).replace("]", "")
my_listp = str(my_listp).replace(" ", "")

#putting the data on a list 
my_list = my_listp.split(",")

l = []
for i in range(0, len(my_list)):
    l.append(my_list[i].split(':'))

In [4]:
# Loops to get names of columns and values 
columns_names = []
for i in range(0,len(l)):
    columns_names.append(l[i][0])

value = []
for i in range(0,len(l)):
    value.append(l[i][1])

In [5]:
#create dataframe from columns 
dfObj = pd.DataFrame(value, index=[columns_names])
df = dfObj.applymap(lambda x: x.replace('"', ''))
df = df.T
df

,x,y,value,category,color
0,1551565.363,1909363.537,1,Corn,#FFD400


## Below is a function with the code above. It takes the values (x, y, year) in the USA Contiguous Albers Equal Area Conic (USGS version) format and returns a dataframe. 

In [6]:
import requests
import pandas as pd
import requests
from bs4 import BeautifulSoup
import timeit

def NASSData(x,y,year):
    url = 'https://nassgeodata.gmu.edu/axis2/services/CDLService/GetCDLValue?year=' + str(year) + '&x=' + str(x) + '&y=' + str(y) 
    response = requests.get(url, verify=False)

    #getting the data from the result section. 
    soup = BeautifulSoup(response.content,'xml')
    table = soup.find_all('Result')[0] 
    
    #cleaning the data.
    reg = str(table)
    reg = re.findall('{(.+?)}', reg)

    my_list = str(reg).replace("'", "")
    my_listp = str(my_list).replace("[", "")
    my_listp = str(my_listp).replace("]", "")
    my_listp = str(my_listp).replace(" ", "")


    my_list = my_listp.split(",")

    l = []
    for i in range(0, len(my_list)):
        l.append(my_list[i].split(':'))
        
# Loops to get values on one list and column names on the other. 
    columns_names = []
    for i in range(0,len(l)):
        columns_names.append(l[i][0])

    value = []
    for i in range(0,len(l)):
        value.append(l[i][1])
        
#create dataframe from columns 
    dfObj = pd.DataFrame(value, index=[columns_names])
    df = dfObj.applymap(lambda x: x.replace('"', ''))
    df = df.T
    
    return(df)

In [7]:
# Example 
NASSData(1551565.363,1909363.537,2015)

,x,y,value,category,color
0,1551565.363,1909363.537,26,DblCropWinWht/Soybeans,#737300


# The function below instead of returning a dataframe, it returns one list. 

In [8]:
import requests
import pandas as pd
import requests
from bs4 import BeautifulSoup

def NASSData_df(x,y,year):
    url = 'https://nassgeodata.gmu.edu/axis2/services/CDLService/GetCDLValue?year=' + str(year) + '&x=' + str(x) + '&y=' + str(y) 
    response = requests.get(url, verify=False)

    #getting the data from the resuls section. 
    soup = BeautifulSoup(response.content,'xml')
    table = soup.find_all('Result')[0] 
    
    #cleaning the data. I will change delete results and things like that 
    reg = str(table)
    reg = re.findall('{(.+?)}', reg)

    my_list = str(reg).replace("'", "")
    my_listp = str(my_list).replace("[", "")
    my_listp = str(my_listp).replace("]", "")
    my_listp = str(my_listp).replace(" ", "")


    my_list = my_listp.split(",")

    l = []
    for i in range(0, len(my_list)):
        l.append(my_list[i].split(':'))

    value = []
    for i in range(0,len(l)):
        value.append(l[i][1])
    return(value)

In [9]:
#Getting for data for one location between 2010 and 2017
p = []
for i in range(2010, 2018):
    casa = NASSData_df(-2172422,1968187,i)
    p.append(casa)
p

[['-2172422.0', '1968187.0', '69', '"Grapes"', '"#704489"'],
 ['-2172422.0', '1968187.0', '69', '"Grapes"', '"#704489"'],
 ['-2172422.0', '1968187.0', '69', '"Grapes"', '"#704489"'],
 ['-2172422.0', '1968187.0', '69', '"Grapes"', '"#704489"'],
 ['-2172422.0', '1968187.0', '69', '"Grapes"', '"#704489"'],
 ['-2172422.0', '1968187.0', '69', '"Grapes"', '"#704489"'],
 ['-2172422.0', '1968187.0', '69', '"Grapes"', '"#704489"'],
 ['-2172422.0', '1968187.0', '69', '"Grapes"', '"#704489"']]

In [10]:
#puting the list in a dataframe
t = pd.DataFrame(p)
t = t.applymap(lambda x: x.replace('"', ''))
t.rename(columns={0: "x", 1: "y", 2: "value", 3: "category", 4: "color"})

,x,y,value,category,color
0,-2172422.0,1968187.0,69,Grapes,#704489
1,-2172422.0,1968187.0,69,Grapes,#704489
2,-2172422.0,1968187.0,69,Grapes,#704489
3,-2172422.0,1968187.0,69,Grapes,#704489
4,-2172422.0,1968187.0,69,Grapes,#704489
5,-2172422.0,1968187.0,69,Grapes,#704489
6,-2172422.0,1968187.0,69,Grapes,#704489
7,-2172422.0,1968187.0,69,Grapes,#704489


## Convertidor de coordinadas 
As we saw above, we need a coordinate converter so we can send latitude and longitude coordinates.

In [12]:
import numpy as np

from shapely.geometry import Point
from functools import partial
import pyproj
from shapely.ops import transform

def converter(x, y):
    point1 = Point(x, y)
    project = partial(
        pyproj.transform,
        #takes latitude and longitude
        pyproj.Proj(init='epsg:4326'),
        #makes the format needed for CropScape
        pyproj.Proj(init='epsg:5071')) 

    point2 = transform(project, point1)
    coordinates_array = np.asarray(point2)
    coordinadas = np.around(coordinates_array, decimals=0)
    return(coordinadas)

In [ ]:
#example
values = converter(-121.3627, 38.1345)
values

In [ ]:
#Testing the two functions together
values = converter(-122.44493, 38.4621)
NASSData(values[0],values[1],2015)

# Now let's say we want to get the data for one location for a range of years.

In [ ]:
import urllib3
# I am disabling the annoying warning signs. 
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

lugares = []
ys = []

for year, r in zip(range(2007, 2019), range(2007, 2019)):
    ys.append(r)
    lugares.append(NASSData_df(-2176010, 1970005, year))
    
#cleaning the data
t = pd.DataFrame(lugares, ys)
t = t.applymap(lambda x: x.replace('"', ''))
t = t.rename(columns={0: "x", 1: "y", 2: "value", 3: "category", 4: "color"})

t

### Function takes latitude and longtitude coordinates and year(s).  It returns a dataframe  with the following columns: year,x, y, value, category, color, Log, and Lat.

In [ ]:
def CropScape(log, lat, yr_Start, yr_End):
    
    lugares = []
    yr = yr_Start
    yr1 = yr_End+1
    ys = []
    
    valores = converter(log, lat)

    for year, r in zip(range(yr, yr1), range(yr, yr1)):
        ys.append(r)
        lugares.append(NASSData_df(valores[0],valores[1], year))
        df_size = len(lugares)
        
    #cleaning the data
    t = pd.DataFrame(lugares, ys)
    t = t.applymap(lambda x: x.replace('"', ''))
    t = t.rename(columns={0: "x", 1: "y", 2: "value", 3: "category", 4: "color"})
    t['Log'] = log
    t['Lat'] = lat 
    return(t)

### I will test it now. <br /> Lat = -121.3003 and Log = 38.0605  <br /> Years: 2011 to 2018

In [ ]:
CropScape(-121.3003, 38.0605, 2011, 2018)

# Getting data for the same location but only for one year.

In [ ]:
CropScape(-121.3003, 38.0605, 2017, 2017)